In [9]:
import pandas as pd
import os

!mkdir resulting_data

def write(text, path):
    f = open(path, 'w')
    f.write(text)
    f.close()

langs = ['DE', 'ID', 'ZH', 'FR', 'RU', 'TR', 'ES']
for lang in langs:
    source_path = f'mturk/samples/data_{lang}.csv'
    target_path = f'resulting_data/{lang}'
    os.makedirs(target_path + '/gold', exist_ok = True)
    os.makedirs(target_path + '/pred_BERT', exist_ok = True)
    os.makedirs(target_path + '/pred_PG', exist_ok = True)
    
    data = pd.read_csv(source_path)
    for idx, row in data.iterrows():
        _, id, type1, type2, model = row['id'].split('-')
        if type1 == 'REFERENCE':
            write(row['text1'], f'{target_path}/gold/{id}')
        elif type1 == 'SYSTEM':
            if model == 'PG':
                write(row['text1'], f'{target_path}/pred_PG/{id}')
            elif model == 'BERT':
                write(row['text1'], f'{target_path}/pred_BERT/{id}')
                
        if type2 == 'REFERENCE':
            write(row['text2'], f'{target_path}/gold/{id}')
        elif type2 == 'SYSTEM':
            if model == 'PG':
                write(row['text2'], f'{target_path}/pred_PG/{id}')
            elif model == 'BERT':
                write(row['text2'], f'{target_path}/pred_BERT/{id}')

In [23]:
import pandas as pd
langs = ['DE', 'ID', 'ZH', 'FR', 'RU', 'TR', 'ES', 'EN']

for lang in langs:
    focus_path = f'mturk/annotation_result/{lang}/human_focus_final.csv'
    coverage_path = f'mturk/annotation_result/{lang}/human_coverage_final.csv'
    target_path = f'resulting_data/{lang}/score.csv'
    
    df_foc = pd.read_csv(focus_path)
    df_cov = pd.read_csv(coverage_path)
    df_score = pd.DataFrame(columns=['id', 'model', 'focus', 'coverage'])
    for idx, row in df_foc.iterrows():
        foc = row['score']
        cov = df_cov.loc[(df_cov['id'] == row['id']) & (df_cov['model'] == row['model'])]['score']
        assert cov.shape[0] == 1
        cov = cov.values[0]
        df_score = df_score.append({'id': row['id'], 'model': row['model'], \
                                   'focus': foc, 'coverage': cov}, ignore_index=True)
    df_score = df_score.sort_values(by=['id', 'model'], ignore_index=True)
    df_score.to_csv(target_path, index=False)    